In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Input, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

C:\Users\Tanvir\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df=pd.read_csv('dataprep.csv',encoding='latin-1')
df.head()

Label                                          Statement
0     H  @Grumpy_P_Sloth @deanesmay feminists argue for...
1     H  1948Army of England helped the Jews to occupy ...
2     H  Dutch Leader Says Europe to collapse In 6 Week...
3     H  RT @__DeLay: The next day the Romans and the J...
4     H  RT @Bakersman_Joe: When Hitler Invited The Jew...

In [3]:
x=df.Statement
y=df.Label

In [4]:
le=LabelEncoder()
y=le.fit_transform(y)
y= y.reshape(-1,1)

In [5]:
x,y=shuffle(x,y,random_state=20)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=2)

In [6]:
maximum_word=1000
maximum_length=150


token=Tokenizer(num_words=maximum_word)
token.fit_on_texts(x_train)
sequences=token.texts_to_sequences(x_train)
sequence_matrix=sequence.pad_sequences(sequences,maxlen=maximum_length)

print(sequences)

sequence_matrix.shape

[[32, 401, 32, 25, 21, 72, 17, 6, 796, 87, 69, 32], [61, 10, 1, 2, 195, 8, 299, 131, 270, 111], [109, 12], [15, 104, 1, 2, 42, 546, 48, 481], [63, 146, 300, 16, 1, 91, 1, 87, 58, 10, 1, 2, 15, 104, 14, 3, 4, 536, 509], [1, 196, 734, 17, 42, 402, 9, 3, 4], [5, 1, 72, 55, 35, 31, 20, 35, 31, 30, 679, 246, 186, 9, 3, 4], [9, 3, 4, 80, 320, 962, 16, 48, 9, 3, 4], [52, 1, 995, 35, 63, 1, 271, 1, 126, 1, 2], [11, 8, 83, 27, 1, 123, 63, 7, 101], [320, 1, 2, 25, 23, 23, 272, 53, 290, 963, 32], [175, 132, 10, 1, 2, 10, 1, 504, 10, 16, 25], [136, 33, 66, 61, 94, 50, 1, 2, 145, 344, 22, 145, 344, 552], [5, 52, 1, 2, 6, 29, 1, 80, 176, 128, 13, 26, 91, 14, 3, 4], [59, 30, 772, 50, 12, 75, 1, 2], [142, 773, 570, 33, 1, 845, 67, 6, 1, 2, 48, 46, 15, 202, 272, 11, 8, 376], [5, 365, 66, 23, 54, 177, 63, 1, 2, 695, 151, 10, 1, 927], [5, 9, 3, 4], [5, 874, 7, 25, 12, 21, 168, 9, 3, 4], [10, 6, 104, 1, 183, 1, 2, 7, 1, 754, 56, 22, 139, 6, 27, 48, 337, 170], [5, 1, 2, 121, 55, 78, 15, 121, 56, 199, 1, 9,

(15270, 150)

In [9]:
def create_model():
    
    inputs = Input(name='inputs',shape=[maximum_length])
    layer = Embedding(maximum_word,50,input_length=maximum_length)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(units=128,activation='relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=1,activation='softmax')(layer)
    layer = Dense(512)(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.3)(layer)
    layer = Dense(512)(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.3)(layer)
    layer = Dense(20)(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    model.compile(loss='binary_crossentropy',optimizer='nadam',metrics=['accuracy'])
    #    model.summary()
    return model

In [10]:
model=KerasClassifier(build_fn=create_model)
#tuning batch size and epochs
#others like optimizer algo, learning rate, momentum, no of lstm_units can be tuned
epochs=[2,4,6]
batch_size=[32,64,128]
param_grid=dict(epochs=epochs,batch_size=batch_size)

## fold cross validation
grid=GridSearchCV(estimator=model,param_grid=param_grid,scoring='accuracy')

grid.fit(sequence_matrix,y_train)

print(grid.best_score_,grid.best_params_)#returns the best cross validation score and parameters
# gave the result 0.9844460397224216 {'batch_size': 128, 'epochs': 4}

NameError: name 'Activation' is not defined

In [ ]:
#train the model with the best epoch and and batch size

inputs = Input(name='inputs',shape=[maximum_length])
layer = Embedding(maximum_word,50,input_length=maximum_length)(inputs)
layer = LSTM(256)(layer)
layer = LSTM(128)(layer)
layer = LSTM(64)(layer)
layer = LSTM(32)(layer)
layer = Dense(units=256,activation='relu')(layer)
#layer = Dropout(0.5)(layer)
layer = Dense(units=1,activation='softmax')(layer)
model = Model(inputs=inputs,outputs=layer)
model.compile(loss='binary_crossentropy',optimizer='nadam',metrics=['accuracy'])
#run the model with batch_size:128 and epochs 4
model.fit(sequence_matrix,y_train,batch_size=128,epochs=4)

#test performance on test set
test_sequences = token.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=maximum_length)

loss_and_accuracy=model.evaluate(test_sequences_matrix,y_test)

print(loss_and_accuracy)